In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [1]:
!git clone https://github.com/minjae0501/hand_tracing.git

Cloning into 'hand_tracing'...
remote: Enumerating objects: 8709, done.
remote: Counting objects: 100% (2405/2405), done.
remote: Compressing objects: 100% (2225/2225), done.
remote: Total 8709 (delta 33), reused 2399 (delta 28), pack-reused 6304
Receiving objects: 100% (8709/8709), 846.74 MiB | 25.50 MiB/s, done.
Resolving deltas: 100% (101/101), done.
Updating files: 100% (9211/9211), done.


In [12]:
import os
import numpy as np

# set random seed at 42
np.random.seed(42)

# BASIC CONFIG

data_dir = '/content/hand_tracing/yolo_block/data'
train_dir = os.path.join(data_dir, 'train/')
val_dir = os.path.join(data_dir, 'val/')
test_dir = os.path.join(data_dir, 'test/')

# get all images inside the first directory in train_dir
train_img_dir = os.path.join(train_dir, 'images')
img_list = os.listdir(train_img_dir)

# Ensure the list is sorted to maintain the order
img_list.sort()

# Split the images into sets of 3
img_sets = [img_list[i:i+3] for i in range(0, len(img_list), 3)]

# Shuffle and split the sets into 8:2 ratio
np.random.shuffle(img_sets)

split_index = int(len(img_sets) * 0.8)
train_img_sets = img_sets[:split_index]
val_img_sets = img_sets[split_index:]

# Flatten the lists to get individual image names
train_img_names = [img for set in train_img_sets for img in set]
val_img_names = [img for set in val_img_sets for img in set]

print('TOTAL SETS: ', len(img_sets))
print('TRAIN SETS: ', len(train_img_sets))
print('VAL SETS: ', len(val_img_sets))
print('TRAIN IMAGES: ', len(train_img_names))
print('VAL IMAGES: ', len(val_img_names))

# Move validation images to val_dir + 'images'
for img_name in val_img_names:
    os.rename(os.path.join(train_img_dir, img_name), os.path.join(val_dir, 'images/', img_name))

# Move corresponding label files for validation images to val_dir + 'labels'
for img_name in val_img_names:
    label_name = img_name.split('.')[0] + '.txt'  # Assuming image and label files share the same base name
    os.rename(os.path.join(train_dir, 'labels/', label_name), os.path.join(val_dir, 'labels/', label_name))

TOTAL SETS:  41
TRAIN SETS:  32
VAL SETS:  9
TRAIN IMAGES:  94
VAL IMAGES:  27


In [20]:
# BASIC CONFIG (필요한 파라미터 값 여기서 설정)

data_dir = '/content/hand_tracing/yolo_block/data/'
model = 'yolov8n.pt'  # pretrained yolov8n

classes = ['blue_bridge', 'blue_half_circle', 'blue_long_rectangle', 'blue_quarter_of_circle', 'green_circle', 'green_cube', 'hand', 'orange_circle', 'orange_cube', 'red_circle', 'red_cube', 'red_thin_rectangle', 'triangle', 'yellow_bridge', 'yellow_cube', 'yellow_half_circle']  # 라벨명
yaml_file = 'block_data.yaml'  # yaml 파일명

training_epoch = 100
patience = 30  # 지정한 epoch 만큼 성능이 개선이 안되면 조기종료.
batch = 32  # batch size (dafault)
imgsz = 640  # input batch size (default)
optimizer = 'Adam'


In [21]:
!nvidia-smi

Fri Dec 29 08:04:24 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0              28W /  70W |    163MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### ULTRALYTICS SETUP

In [22]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.231 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (8 CPUs, 51.0 GB RAM, 27.9/166.8 GB disk)


In [23]:
from ultralytics import YOLO
import os
import yaml
from IPython.display import display, Image
from IPython import display

display.clear_output()
!yolo checks

Ultralytics YOLOv8.0.231 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (8 CPUs, 51.0 GB RAM, 27.9/166.8 GB disk)

OS                  Linux-6.1.58+-x86_64-with-glibc2.35
Environment         Colab
Python              3.10.12
Install             pip
RAM                 50.99 GB
CPU                 Intel Xeon 2.20GHz
CUDA                12.1

matplotlib          ✅ 3.7.1>=3.3.0
numpy               ✅ 1.23.5>=1.22.2
opencv-python       ✅ 4.8.0.76>=4.6.0
pillow              ✅ 9.4.0>=7.1.2
pyyaml              ✅ 6.0.1>=5.3.1
requests            ✅ 2.31.0>=2.23.0
scipy               ✅ 1.11.4>=1.4.1
torch               ✅ 2.1.0+cu121>=1.8.0
torchvision         ✅ 0.16.0+cu121>=0.9.0
tqdm                ✅ 4.66.1>=4.64.0
psutil              ✅ 5.9.5
py-cpuinfo          ✅ 9.0.0
thop                ✅ 0.1.1-2209072238>=0.1.1
pandas              ✅ 1.5.3>=1.1.4
seaborn             ✅ 0.12.2>=0.11.0


### Create yaml file

In [24]:
# python: dictionary <----------> yaml
data = {
    'train': data_dir + 'train/',
    'val': data_dir + 'val/',
    'test': data_dir + 'test/',
    'nc': 16,
    'names': classes
}

os.makedirs(data_dir, exist_ok = True) # directory to save

with open(data_dir + yaml_file, 'wt') as fw:
    yaml.dump(data, fw)


In [25]:
with open(data_dir + yaml_file, 'rt') as fr:
    d = yaml.safe_load(fr)
    print(type(d))
    print(d)

<class 'dict'>
{'names': ['blue_bridge', 'blue_half_circle', 'blue_long_rectangle', 'blue_quarter_of_circle', 'green_circle', 'green_cube', 'hand', 'orange_circle', 'orange_cube', 'red_circle', 'red_cube', 'red_thin_rectangle', 'triangle', 'yellow_bridge', 'yellow_cube', 'yellow_half_circle'], 'nc': 16, 'test': '/content/hand_tracing/yolo_block/data/test/', 'train': '/content/hand_tracing/yolo_block/data/train/', 'val': '/content/hand_tracing/yolo_block/data/val/'}


### Model Training

In [26]:
import torch

if torch.cuda.is_available() == True:
    device = 'cuda:0'
else:
    device = 'cpu'

print(device)

cuda:0


In [27]:
model = YOLO(model).to(device)  # pretrained yolov8n

model.train(data = data_dir + yaml_file,
            epochs = training_epoch,
            patience = patience, # 지정한 epoch 만큼 성능이 개선이 안되면 조기종료.
            batch = batch, # batch size (dafault)
            imgsz = imgsz, # input batch size (default)
            optimizer = optimizer)

engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/hand_tracing/yolo_block/data/block_data.yaml, epochs=100, time=None, patience=30, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscript, keras=False, optimize=False

100%|██████████| 755k/755k [00:00<00:00, 5.13MB/s]


Overriding model.yaml nc=80 with nc=16

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

train: Scanning /content/hand_tracing/yolo_block/data/train/labels... 94 images, 0 backgrounds, 0 corrupt: 100%|██████████| 94/94 [00:00<00:00, 1084.46it/s]

train: New cache created: /content/hand_tracing/yolo_block/data/train/labels.cache


...

    100/100      4.59G     0.8826     0.4348      1.102        342        640: 100%|██████████| 3/3 [00:00<00:00,  3.68it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  5.18it/s]


                   all         27        319      0.956       0.97      0.989      0.719

100 epochs completed in 0.049 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 6.3MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics YOLOv8.0.231 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3008768 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  4.27it/s]


                   all         27        319      0.958      0.971      0.989      0.721
           blue_bridge         27         25      0.965          1      0.995      0.758
      blue_half_circle         27         19      0.993          1      0.995      0.753
   blue_long_rectangle         27         14      0.987          1      0.995      0.642
blue_quarter_of_circle         27          9      0.803          1      0.995      0.747
          green_circle         27         25      0.968          1      0.995      0.729
            green_cube         27         26          1      0.982      0.995      0.799
                  hand         27         26       0.96          1      0.995      0.657
         orange_circle         27         19          1      0.905      0.995      0.769
           orange_cube         27         18          1       0.99      0.995      0.671
            red_circle         27         20          1      0.912       0.99      0.707
              red_cub

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x79a0efbffc70>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044

### Model Evaluation

In [28]:
best_model_path = '/content/runs/detect/train3/weights/best.pt'

best_model = YOLO(best_model_path).to(device)
metrics = best_model.val()

for label, ap in zip(classes, metrics.box.maps):
    print(label, ':', ap)

Model summary (fused): 168 layers, 3008768 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/hand_tracing/yolo_block/data/val/labels.cache... 27 images, 0 backgrounds, 0 corrupt: 100%|██████████| 27/27 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:03<00:00,  1.92s/it]


                   all         27        319      0.958      0.971      0.989      0.719
           blue_bridge         27         25      0.966          1      0.995      0.747
      blue_half_circle         27         19      0.993          1      0.995      0.753
   blue_long_rectangle         27         14      0.987          1      0.995      0.622
blue_quarter_of_circle         27          9      0.804          1      0.995      0.747
          green_circle         27         25      0.968          1      0.995      0.734
            green_cube         27         26          1      0.982      0.995      0.799
                  hand         27         26      0.959          1      0.995      0.652
         orange_circle         27         19          1      0.905      0.995      0.763
           orange_cube         27         18          1       0.99      0.995      0.671
            red_circle         27         20          1      0.912       0.99      0.712
              red_cub